## VISUALIZATION PROJECT Geospatial Business Intelligence (BI)


In [6]:
#Tras iniciar servidor de DB mongo, importo módulo 'pymongo'.
import pymongo
import pandas as pd
#Asigno variable al cliente de DB y a la DB.
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [7]:
#Verifico que se han importado todos los documentos.
cursor = db.companies.find()
data = list(cursor)
print(len(data))

18801


In [3]:
def q_qty_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return len(list(map(lambda r:r,list(q))))
atr1 = 'category_code'
var1 = 'web'
print(q_qty_1(atr1,var1))

3787


In [9]:
def q_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return list(map(lambda r:[(r['name']),(r['offices'])],list(q)))
atr1 = 'category_code'
var1 = 'web'
display(q_1(atr1,var1))

[['Wetpaint',
  [{'description': '',
    'address1': '710 - 2nd Avenue',
    'address2': 'Suite 1100',
    'zip_code': '98104',
    'city': 'Seattle',
    'state_code': 'WA',
    'country_code': 'USA',
    'latitude': 47.603122,
    'longitude': -122.333253},
   {'description': '',
    'address1': '270 Lafayette Street',
    'address2': 'Suite 505',
    'zip_code': '10012',
    'city': 'New York',
    'state_code': 'NY',
    'country_code': 'USA',
    'latitude': 40.7237306,
    'longitude': -73.9964312}]],
 ['Postini',
  [{'description': None,
    'address1': '959 Skyway Road, Suite 200',
    'address2': None,
    'zip_code': '94070',
    'city': 'San Carlos',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.506885,
    'longitude': -122.247573}]],
 ['Geni',
  [{'description': 'Headquarters',
    'address1': '9229 W. Sunset Blvd.',
    'address2': '',
    'zip_code': '90069',
    'city': 'West Hollywood',
    'state_code': 'CA',
    'country_code': 'USA',
    'lat

In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

In [ ]:
def convertRestaurant(restaurant):
    return {
        "name": restaurant["name"],
        "cuisine":restaurant["cuisine"],
        "position": toGeoJSON(restaurant["address"]["coord"])
    }


In [5]:
cursor = 
df = pd.DataFrame(data)
df.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,...,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]
1,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,...,1-888-204-3539,"[{'name': 'Zoho Office Suite', 'permalink': 'z...",[],"[{'is_past': False, 'title': 'CEO and Founder'...",[],"zoho, officesuite, spreadsheet, writer, projec...",$0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{'embed_code': '<object width=""430"" height=""2..."
2,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,...,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
3,52cdef7c4bab8bd675297d8f,None,[],None,http://feeds.feedburner.com/omnidrive,http://www.omnidrive.com/blog,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",Sun May 27 03:25:32 UTC 2007,http://www.crunchbase.com/company/omnidrive,...,660-675-5052,"[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]",[],"[{'is_past': True, 'title': 'Co-founder', 'per...",[],"storage, sharing, edit, online",$800k,Nomadesk,Tue Jul 02 22:48:04 UTC 2013,[]
4,52cdef7c4bab8bd675297d90,"{'price_amount': 625000000, 'price_currency_co...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,...,888.584.3150,"[{'name': 'Postini Solutions', 'permalink': 'p...",[],"[{'is_past': False, 'title': 'Board', 'person'...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[]
